In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

Generate a timetable with less confliction

In [ ]:
import random

In [ ]:
import time
import datetime

In [ ]:
from termcolor import colored

# Prepare the data structure 

In [ ]:
# Slot = Index , Day , Hour
class SlotProcessor:
    def __init__(self, inputString):
        stringParts = inputString.split(",")
        cursorPosition = 0
        self.id = stringParts[cursorPosition].strip()
        cursorPosition += 1
        self.day = stringParts[cursorPosition].strip()
        cursorPosition += 1
        self.time = stringParts[cursorPosition].strip()
        
    def getSlotInfo(self):
        print('Index: {} | Day: {} | Hour:{}'.format(self.id, self.day, self.time))

In [ ]:
# Room = Index , Name , Capacity
class RoomProcessor:
    def __init__(self, inputString):
        stringParts = inputString.split(",")
        cursorPosition = 0

        self.id = stringParts[cursorPosition].strip()
        cursorPosition += 1

        self.name = stringParts[cursorPosition].strip()

    def getRoomInfo(self):
        print('Index: {} | name: {}'.format(self.id, self.name))

In [ ]:
#  Course = Index , Name , Capacity , Arrangement , InstructorIndex
class CourseProcessor:
    def __init__(self, inputString):
        stringParts = inputString.split(",")
        cursorPosition = 0

        self.id = stringParts[cursorPosition].strip()
        cursorPosition += 1

        self.name = stringParts[cursorPosition].strip()
        cursorPosition += 1

        self.capacity = stringParts[cursorPosition].strip()
        cursorPosition += 1

        self.arrangement = stringParts[cursorPosition].strip()
        cursorPosition += 1

        self.instructorIndex = int(stringParts[cursorPosition].strip())

    def getCourseInfo(self):
        print("Course ID: {} | Course Name: {} | Roomtype: {} | Arrangement: {} | InstructorIndex: {}"\
              .format(self.id, self.name, self.capacity, self.arrangement, self.instructorIndex))
    
    def getCourseName(self):
        return self.name
    def getCourseInstructorIndex(self):
        return int(self.instructorIndex)
     

In [ ]:
# Instructor = Index , Name 
class InstructorProcessor:
    def __init__(self, inputString):
        stringParts = inputString.split(",")
        cursorPosition = 0

        self.id = stringParts[cursorPosition].strip()
        cursorPosition += 1

        self.name = stringParts[cursorPosition].strip()
        cursorPosition += 1

        #for i in range(cursorPosition, cursorPosition + numberOfUnpreferreds):
            #self.unpreferreds.append(int(stringParts[i]))
     
    def getInstructorIndex(self):
        return int(self.id)
    
    def getInstructorName(self):
        return self.name
    
    def getInstructorInfo(self):
        print("Instructor ID: {} | Instructor Name: {}".format(self.id, self.name))
    
    # Highlight function getInstructorNameViaIndex(instructorIndex)
    def getInstructorNameViaIndex(self, instructorIndex):
        if int(self.id) == instructorIndex:
            print (self.name) 
        else:
            print('There is no instructor with this instructor index')


In [ ]:
slots = []
rooms = []
courses = []
instructors = []
with open('input4.txt') as fp:
    for line in fp:
        if line.startswith("#"):
        # ignore comments in input file
            continue
        lineSections = line.split("=")
        # print(lineSections)
        if lineSections[0].strip() == "Slot":
            #1 , one    ,  8:00
            # print(lineSections[1])
            slots.append(SlotProcessor(lineSections[1]))
            # slots
        elif lineSections[0].strip() == "Room":
            rooms.append(RoomProcessor(lineSections[1]))
        elif lineSections[0].strip() == "Course":
            courses.append(CourseProcessor(lineSections[1]))
        elif lineSections[0].strip() == "Instructor":
            instructors.append(InstructorProcessor(lineSections[1]))
print('number of slots: {}, number of rooms: {}, number of courses: {}, number of instructors: {}'.\
     format(len(slots), len(rooms), len(courses), len(instructors)))

Fill the timetable with name of the course and name of the Instructor

# Define the object to be added into each slot

In [ ]:
class courseInstructor:
    def __init__(self, an_item_in_courses, an_item_in_instructors):
        self.course = an_item_in_courses
        # an_item_in_courses will determine which an_item_in_instructors
        
        self.instructor = an_item_in_instructors
    """
    def printCourseInstructorInfo(self):
        print('course name: {} | instructor name: {}'.\
              format(self.course.getCourseName(),\
                     self.instructor.getInstructorNameViaIndex(self.course.getCourseInstructorIndex() )))
    """

    def printCourseInstructorInfo(self): # Don't forget situation that 2 teachers may have the same name.
        return('course name: {} | instructor name: {}'.\
              format(self.course.getCourseName(),\
                     self.instructor.getInstructorName()))    

`ScheduleTable.createSchedule()` is supposed to fill the 40 courses for class one.

Check whether this course can be added into this slot or not.

<span style = "font-family:Times New Roman ; font-size: 2em; color: green"> 
Put emphasis on the engine!!!
</span>


In [ ]:
# Extract condition logic out from class Schedule
def IsInstructorAvailable(which_class, CurrentInstructor, ExistedSlots, which_slot):
    if which_class == 1:
        return False
    else:
        i = 1
        a = 0
        while i < which_class:          
            if CurrentInstructor == ExistedSlots[which_slot - 40 * i][0].instructor.getInstructorIndex():
                a += 1
            else:
                a += 0
            
            i += 1
        if a == 0:
            return False
        else:
            return True


In [ ]:
class Schedule:
    def __init__(self, slots, rooms, courses, instructors):
        self.schedule_slots = slots
        self.schedule_rooms = rooms
        self.schedule_courses = courses
        self.schedule_instructors = instructors
        self.classes = []
        self.slots = []
        self.fitness = 0.0
        self.number_of_courses_for_a_class_per_week = len(self.schedule_slots)
        self.weekday = 5
        self.number_of_courses_for_a_class_per_day=self.number_of_courses_for_a_class_per_week/self.weekday

        for i in range(0, len(slots) * len(rooms)): # range(0, 240)
            self.slots.append([]) #Change self.slots from [] to [[],[],[]...]
            
    def inspectSchedule(self):
        if self.slots[0] == []:
                print('Wake up, before looking up the schedule, create it at first!')
        else:
                
            for i in range(0, len(slots) * len(rooms)):
                print( 'CourseInstructorInfo: {} | {}'\
                      .format(self.slots[i][0].printCourseInstructorInfo(), i))
            
    def createSchedule(self):
        number_of_courses_for_all_classes_per_week = len(self.slots) # 240
        number_of_course_types = len(self.schedule_courses)/len(self.schedule_rooms) 
        counter_the_number_of_arranged_course = 1
        initial_choice = [list(range(0,40)), list(range(40,80)),\
                          list(range(80,120)), list(range(120,160)),\
                          list(range(160,200)), list(range(200,240))]
        
        for course in self.schedule_courses:           
            slotContent = courseInstructor(course, instructors[course.getCourseInstructorIndex() - 1])
            classIndex=(int(course.id)+15)//number_of_course_types # Highlights
            InstructorIndex = course.getCourseInstructorIndex()
            
            repeated_times_for_a_course = int(course.arrangement[1])
            j = 0 
            while j < repeated_times_for_a_course:
                abondon_choice = random.choice(initial_choice[int(classIndex - 1) ]) # A little confused, I think classIndex - 1 is supposed to be int.
                while IsInstructorAvailable(classIndex, InstructorIndex, self.slots, abondon_choice):
                    print('WRONG! course {}, place it to the {}th slot'.format(course.getCourseName(), abondon_choice))
                    print(colored('Confliction appear, regenerate','red'))
                    abondon_choice = random.choice(initial_choice[int(classIndex - 1)])
                        
                self.slots[abondon_choice].append(slotContent)
                print('OKAY! The {}th course :{}, place it to the {}th slot'.\
                      format(counter_the_number_of_arranged_course,course.getCourseName(), abondon_choice))
                counter_the_number_of_arranged_course += 1
                print(self.slots[abondon_choice][0].printCourseInstructorInfo()) # Different from Version1
                initial_choice[int(classIndex - 1)].remove(abondon_choice)
                j = j + 1    

# Test class `Schedule` locally

In [ ]:
ScheduleTable = Schedule(slots, rooms, courses, instructors )

## Something is wrong.

In [29]:
begin_time = time.time()
i = 0
while i < 1:
    ScheduleTable.createSchedule()
    i +=1 
cost_time = time.time() -begin_time

OKAY! The 1th course :语文, place it to the 1th slot
course name: 物理 | instructor name: 吴发祥
OKAY! The 2th course :语文, place it to the 23th slot
course name: 数学 | instructor name: 马云芝
OKAY! The 3th course :语文, place it to the 19th slot
course name: 数学 | instructor name: 马云芝
OKAY! The 4th course :语文, place it to the 16th slot
course name: 生物 | instructor name: 生物老师1
OKAY! The 5th course :语文, place it to the 22th slot
course name: 化学 | instructor name: 张文云
OKAY! The 6th course :数学, place it to the 34th slot
course name: 政治 | instructor name: 政治老师1
OKAY! The 7th course :数学, place it to the 10th slot
course name: 生物 | instructor name: 生物老师1
OKAY! The 8th course :数学, place it to the 3th slot
course name: 生物 | instructor name: 生物老师1
OKAY! The 9th course :数学, place it to the 31th slot
course name: 地理 | instructor name: 地理老师1
OKAY! The 10th course :数学, place it to the 27th slot
course name: 英语 | instructor name: 罗迎春
OKAY! The 11th course :英语, place it to the 28th slot
course name: 化学 | instructor

limit_output extension: Maximum message size of 10000 exceeded with 23791 characters

In [30]:
cost_time

0.028477907180786133

In [25]:
begin_time = time.time()
i = 0
while i < 10:
    ScheduleTable.createSchedule()
    i +=1 
cost_time = time.time() -begin_time



Confliction appear, regenerate
WRONG! course 劳技, place it to the 174th slot
Confliction appear, regenerate
WRONG! course 劳技, place it to the 174th slot
Confliction appear, regenerate
WRONG! course 劳技, place it to the 174th slot
Confliction appear, regenerate
WRONG! course 劳技, place it to the 174th slot
Confliction appear, regenerate
WRONG! course 劳技, place it to the 174th slot
Confliction appear, regenerate
WRONG! course 劳技, place it to the 174th slot
Confliction appear, regenerate
WRONG! course 劳技, place it to the 174th slot
Confliction appear, regenerate
WRONG! course 劳技, place it to the 174th slot
Confliction appear, regenerate
WRONG! course 劳技, place it to the 174th slot
Confliction appear, regenerate
WRONG! course 劳技, place it to the 174th slot
Confliction appear, regenerate
WRONG! course 劳技, place it to the 174th slot
Confliction appear, regenerate
WRONG! course 劳技, place it to the 174th slot
Confliction appear, regenerate
WRONG! course 劳技, place it to the 174th slot
Confliction

limit_output extension: Maximum message size of 10000 exceeded with 84956 characters

KeyboardInterrupt: 

**Turn the condition in to a function, like `IsInstructorAvailable`**. The logic in above snippet above is frank but not smart.

In [ ]:
ScheduleTable.inspectSchedule()